In [1]:
import json 
import pandas as pd

# load payload
with open("../data/payload_mimic3.json", "r") as f:
    loaded_payload = json.load(f)

loaded_df = pd.DataFrame(loaded_payload)

In [7]:
loaded_df['subject_id'].value_counts()

subject_id
41976    15
10088     3
44083     3
10059     2
10119     2
         ..
43927     1
44154     1
44212     1
44222     1
44228     1
Name: count, Length: 100, dtype: int64

In [15]:
loaded_df[loaded_df['subject_id'] == 41976].sort_values('hadm_id', ascending=False).head(1)

,subject_id,hadm_id,admittime,dischtime,admission_type,insurance,language,marital_status,ethnicity,gender,...,creatinina,hemoglobina,potassio,sodio,creatinina_uom,hemoglobina_uom,potassio_uom,sodio_uom,notas_24h_resumo,prescricoes
82,41976,180546,7301011740000,7301628240000,EMERGENCY,Medicare,SPAN,MARRIED,HISPANIC/LATINO - PUERTO RICAN,M,...,0.4,9.4,3.4,137.0,mg/dL,g/dL,mEq/L,mEq/L,None,"[{'drug': 'Fentanyl Patch', 'drug_type': 'MAIN..."


In [41]:
def get_most_recent_admission(df: pd.DataFrame, subject_id: int) -> dict:
    subject_df = df[df['subject_id'] == subject_id]
    most_recent_admission = subject_df.sort_values('hadm_id', ascending=False).head(1)
    return most_recent_admission.iloc[0].to_dict()

def format_admission_info(admission: dict) -> str:
    all_prescriptions = []
    for prescription in admission['prescricoes']:
        presc_info = (
            f"  - Drug: {prescription['drug']}, "
            f"Type: {prescription['drug_type']}, "
            f"Dose: {prescription['dose_val']} {prescription['dose_unit']}, "
            f"Form: {prescription['form']}, "
            f"Route: {prescription['route']}, "
            f"Start: {prescription['starttime']}, "
            f"Stop: {prescription['stoptime']}"
            "--------------------------------"
        )
        all_prescriptions.append(presc_info)
        
    all_prescriptions_str = "\n".join(all_prescriptions)
    
    info = (
        f"Subject ID: {admission['subject_id']}\n"
        f"Admission ID: {admission['hadm_id']}\n"
        f"Admission Date: {admission['admittime']}\n"
        f"Discharge Date: {admission['dischtime']}\n"
        f"Insurance: {admission['insurance']}\n"
        f"Age: {admission['age_estimada']}\n"
        f"Gender: {admission['gender']}\n"
        f"Language: {admission['language']}\n"
        f"Marital Status: {admission['marital_status']}\n"
        f"Ethnicity: {admission['ethnicity']}\n"
        f"Creatinine: {admission['creatinina']} {admission['creatinina_uom']}\n"
        f"Hemoglobin: {admission['hemoglobina']} {admission['hemoglobina_uom']}\n"
        f"Potassium: {admission['potassio']} {admission['potassio_uom']}\n"
        f"Sodium: {admission['sodio']} {admission['sodio_uom']}\n"
        f"Notes (24h Summary): {admission['notas_24h_resumo']}\n"
        # f"Prescriptions: {all_prescriptions_str}\n"
        f"----------------------------------------\n"
    )
    return info

def get_last_prescription(admission: dict) -> dict:
    prescriptions = admission['prescricoes']
    last = prescriptions[-1] if prescriptions else None
    return last

In [42]:
admission = get_most_recent_admission(loaded_df, 41976)

In [43]:
get_last_prescription(admission)

{'drug': 'Duloxetine',
 'drug_type': 'MAIN',
 'dose_val': '30',
 'dose_unit': 'mg',
 'form': None,
 'route': 'PO',
 'starttime': '2201-05-12 00:00:00',
 'stoptime': '2201-05-19 00:00:00'}

In [39]:
print(format_admission_info(admission))

Subject ID: 41976
Admission ID: 180546
Admission Date: 7301011740000
Discharge Date: 7301628240000
Insurance: Medicare
Age: None
Gender: M
Language: SPAN
Marital Status: MARRIED
Ethnicity: HISPANIC/LATINO - PUERTO RICAN
Creatinine: 0.4 mg/dL
Hemoglobin: 9.4 g/dL
Potassium: 3.4 mEq/L
Sodium: 137.0 mEq/L
Notes (24h Summary): None
----------------------------------------



In [46]:
admission['prescricoes']

[{'drug': 'Fentanyl Patch',
  'drug_type': 'MAIN',
  'dose_val': '50',
  'dose_unit': 'mcg/hr',
  'form': None,
  'route': 'TP',
  'starttime': '2201-05-12 00:00:00',
  'stoptime': '2201-05-14 00:00:00'},
 {'drug': 'Gabapentin',
  'drug_type': 'MAIN',
  'dose_val': '300',
  'dose_unit': 'mg',
  'form': None,
  'route': 'PO/NG',
  'starttime': '2201-05-12 00:00:00',
  'stoptime': '2201-05-14 00:00:00'},
 {'drug': 'Levothyroxine Sodium',
  'drug_type': 'MAIN',
  'dose_val': '25',
  'dose_unit': 'mcg',
  'form': None,
  'route': 'PO/NG',
  'starttime': '2201-05-12 00:00:00',
  'stoptime': '2201-05-19 00:00:00'},
 {'drug': 'Mirtazapine',
  'drug_type': 'MAIN',
  'dose_val': '15',
  'dose_unit': 'mg',
  'form': None,
  'route': 'PO/NG',
  'starttime': '2201-05-12 00:00:00',
  'stoptime': '2201-05-19 00:00:00'},
 {'drug': 'Insulin',
  'drug_type': 'MAIN',
  'dose_val': '0',
  'dose_unit': 'UNIT',
  'form': None,
  'route': 'SC',
  'starttime': '2201-05-12 00:00:00',
  'stoptime': '2201-05-19

In [47]:
# Analisar os tipos de drogas no dataset
print("📊 ANÁLISE DOS TIPOS DE DROGAS (drug_type)")
print("=" * 50)

# Contar tipos de drogas
drug_types = []
for _, row in loaded_df.iterrows():
    for presc in row['prescricoes']:
        if presc.get('drug_type'):
            drug_types.append(presc['drug_type'])

drug_type_counts = pd.Series(drug_types).value_counts()
print("\n🔢 Frequência dos tipos:")
print(drug_type_counts)

print("\n" + "=" * 50)
print("📋 EXEMPLOS DE CADA TIPO:")
print("=" * 50)

# Mostrar exemplos de cada tipo
for drug_type in drug_type_counts.index[:5]:  # Top 5 tipos
    print(f"\n🏷️  {drug_type}:")
    examples = []
    for _, row in loaded_df.iterrows():
        for presc in row['prescricoes']:
            if presc.get('drug_type') == drug_type and len(examples) < 3:
                examples.append(presc['drug'])
        if len(examples) >= 3:
            break
    
    for i, example in enumerate(examples, 1):
        print(f"   {i}. {example}")

📊 ANÁLISE DOS TIPOS DE DROGAS (drug_type)

🔢 Frequência dos tipos:
MAIN        7709
BASE        2617
ADDITIVE      72
Name: count, dtype: int64

📋 EXEMPLOS DE CADA TIPO:

🏷️  MAIN:
   1. Sodium Chloride 0.9%  Flush
   2. Glipizide
   3. Metoprolol

🏷️  BASE:
   1. Iso-Osmotic Dextrose
   2. NS
   3. D5W

🏷️  ADDITIVE:
   1. Calcium Gluconate
   2. Potassium Chloride
   3. Calcium Gluconate


In [48]:
print("🎯 DIFERENÇAS ENTRE TIPOS DE DROGAS NO MIMIC:")
print("=" * 60)

print("""
🏷️  MAIN (Medicamento Principal):
   • Medicamentos ativos com ação farmacológica específica
   • Drogas que tratam condições médicas diretamente
   • Exemplos: Antibióticos, Anti-hipertensivos, Analgésicos
   • Ex: Metoprolol (beta-bloqueador), Glipizide (antidiabético)

🏷️  BASE (Solução Base/Veículo):
   • Soluções de base para diluição ou hidratação
   • Fluidos intravenosos básicos
   • Servem como veículo para outros medicamentos
   • Ex: Soro fisiológico (NS), Dextrose 5% (D5W)

🏷️  ADDITIVE (Aditivo):
   • Substâncias adicionadas às soluções BASE
   • Eletrólitos, vitaminas, minerais
   • Complementam ou fortalecem as soluções principais
   • Ex: Potássio, Cálcio, Magnésio

💡 EXEMPLO PRÁTICO:
   • BASE: Soro fisiológico 500ml
   • ADDITIVE: + Potássio 20mEq
   • MAIN: Antibiótico (administrado separadamente)
""")

🎯 DIFERENÇAS ENTRE TIPOS DE DROGAS NO MIMIC:

🏷️  MAIN (Medicamento Principal):
   • Medicamentos ativos com ação farmacológica específica
   • Drogas que tratam condições médicas diretamente
   • Exemplos: Antibióticos, Anti-hipertensivos, Analgésicos
   • Ex: Metoprolol (beta-bloqueador), Glipizide (antidiabético)

🏷️  BASE (Solução Base/Veículo):
   • Soluções de base para diluição ou hidratação
   • Fluidos intravenosos básicos
   • Servem como veículo para outros medicamentos
   • Ex: Soro fisiológico (NS), Dextrose 5% (D5W)

🏷️  ADDITIVE (Aditivo):
   • Substâncias adicionadas às soluções BASE
   • Eletrólitos, vitaminas, minerais
   • Complementam ou fortalecem as soluções principais
   • Ex: Potássio, Cálcio, Magnésio

💡 EXEMPLO PRÁTICO:
   • BASE: Soro fisiológico 500ml
   • ADDITIVE: + Potássio 20mEq
   • MAIN: Antibiótico (administrado separadamente)



In [49]:
# Exemplo real das prescrições do paciente separadas por tipo
print("📋 EXEMPLO REAL - Prescrições do Paciente 41976:")
print("=" * 60)

for i, presc in enumerate(admission['prescricoes'], 1):
    drug_type = presc.get('drug_type', 'N/A')
    
    if drug_type == 'MAIN':
        emoji = "💊"
    elif drug_type == 'BASE':
        emoji = "🧪"
    elif drug_type == 'ADDITIVE':
        emoji = "➕"
    else:
        emoji = "❓"
    
    print(f"{emoji} {drug_type}: {presc['drug']}")
    print(f"   Dose: {presc['dose_val']} {presc['dose_unit']}")
    print(f"   Via: {presc['route']}")
    print(f"   Período: {presc['starttime']} → {presc['stoptime']}")
    print()

print("🎯 RESUMO:")
print(f"💊 MAIN: Medicamentos ativos = {len([p for p in admission['prescricoes'] if p.get('drug_type') == 'MAIN'])}")
print(f"🧪 BASE: Soluções veículo = {len([p for p in admission['prescricoes'] if p.get('drug_type') == 'BASE'])}")
print(f"➕ ADDITIVE: Aditivos/eletrólitos = {len([p for p in admission['prescricoes'] if p.get('drug_type') == 'ADDITIVE'])}")

📋 EXEMPLO REAL - Prescrições do Paciente 41976:
💊 MAIN: Fentanyl Patch
   Dose: 50 mcg/hr
   Via: TP
   Período: 2201-05-12 00:00:00 → 2201-05-14 00:00:00

💊 MAIN: Gabapentin
   Dose: 300 mg
   Via: PO/NG
   Período: 2201-05-12 00:00:00 → 2201-05-14 00:00:00

💊 MAIN: Levothyroxine Sodium
   Dose: 25 mcg
   Via: PO/NG
   Período: 2201-05-12 00:00:00 → 2201-05-19 00:00:00

💊 MAIN: Mirtazapine
   Dose: 15 mg
   Via: PO/NG
   Período: 2201-05-12 00:00:00 → 2201-05-19 00:00:00

💊 MAIN: Insulin
   Dose: 0 UNIT
   Via: SC
   Período: 2201-05-12 00:00:00 → 2201-05-19 00:00:00

💊 MAIN: Insulin
   Dose: 32 UNIT
   Via: SC
   Período: 2201-05-12 00:00:00 → 2201-05-19 00:00:00

💊 MAIN: Docusate Sodium (Liquid)
   Dose: 200 mg
   Via: PO/NG
   Período: 2201-05-12 00:00:00 → 2201-05-19 00:00:00

💊 MAIN: Morphine Sulfate (Oral Soln.)
   Dose: 15 mg
   Via: PO/NG
   Período: 2201-05-12 00:00:00 → 2201-05-19 00:00:00

💊 MAIN: Acetylcysteine 20%
   Dose: 1-10 mL
   Via: NEB
   Período: 2201-05-12 00:00: